# Evaluate the performance of a trained model

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
from torch.nn import DataParallel

from vos.models.STM import STM
from vos.algo.stm_train import STMAlgo
from vos.models.EMN import EMN
from vos.algo.emn_train import EMNAlgo
from vos.datasets.DAVIS import DAVIS_2017_TrainVal
from vos.utils.helpers import stack_images, stack_masks

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

Space-time Memory Networks: initialized.


In [2]:
davisroot = "/p300/videoObjSeg_dataset/DAVIS-2017-trainval-480p"
videod = DAVIS_2017_TrainVal(davisroot, mode= "train")
dataloader = DataLoader(videod)
print(len(dataloader))

60


In [3]:
model = DataParallel(STM())
algo = STMAlgo(eval_step_kwargs= dict(Mem_every= 5),)

# model = DataParallel(EMN())
# algo = EMNAlgo(eval_step_kwargs= dict(Mem_every= 5),)

model.cuda()
algo.initialize(model)
model.eval()

DataParallel(
  (module): STM(
    (Encoder_M): Encoder_M(
      (conv1_m): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv1_o): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (res2): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d

## Start evaluating and calculating the mean loss and performance

In [4]:
def run_test():
    loss, IoU = 0.,0.
    IoU_each_frame = []
    for i, data in enumerate(dataloader):
        eval_info, extra_info = algo.eval(i, data)
        loss += eval_info.loss
        IoU += eval_info.IoU
        IoU_each_frame.append(extra_info["IoU_each_frame"])
        print(f"Evaluating at itereation: {i}, ", end= "")
        print(f"IoU_this: {eval_info.IoU}, ", end= "")
        print(f"IoU: {IoU/(i+1)}, ", end= "")
        print(f"IoU_frame_ave: {np.nanmean(np.hstack(IoU_each_frame))} ", end= "")
        print("n_frames: {}, ".format(extra_info["IoU_each_frame"].shape[0]), end= "\r")
    return IoU / (i+1), np.nanmean(np.hstack(IoU_each_frame))

In [ ]:
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200319/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-4/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200319/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-1/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200319/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNEMN/big_objects-True1/b_size-1/run_0"
statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200311/affine0.0-10.0-0.05-0.0/img_res-384,384/NNSTM/big_objects-True3/b_size-4/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200322/synth5-5-0.05-5-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-4/run_0"


IoU = 0
frames_IoU = 0
while IoU < 0.57 and frames_IoU < 0.57:
    state_dict = torch.load(os.path.join(statedict_root, "params.pkl"))
    model_state_dict = state_dict['model_state_dict']
    model.load_state_dict(model_state_dict)
    model.eval()
    IoU, frames_IoU = run_test()
    print("IoU: ", IoU, "frames_IoU: ", frames_IoU)

/root/VideoObjSeg/vos/algo/videoobjseg.py:53: RuntimeWarning: invalid value encountered in true_divide
  IoU = np.sum(gtruth & pred, axis= 1) / np.sum(gtruth | pred, axis= 1)
/root/VideoObjSeg/vos/algo/videoobjseg.py:57: RuntimeWarning: invalid value encountered in true_divide
  accuracy_rate = intersect / np.sum(pred, axis= 1)
/root/VideoObjSeg/vos/algo/videoobjseg.py:58: RuntimeWarning: invalid value encountered in true_divide
  recall_rate = intersect / np.sum(gtruth, axis= 1)


/root/VideoObjSeg/vos/algo/videoobjseg.py:60: RuntimeWarning: invalid value encountered in true_divide
  (accuracy_rate * recall_rate) / (accuracy_rate + recall_rate)


In [ ]:
dst = os.path.join(statedict_root, "params-{:.2f}.pkl".format(IoU * 100))
torch.save(state_dict, dst)